In [1]:
import sqlite3
from keras.models import load_model
from matplotlib.pyplot import imshow
import tqdm.notebook as tqdm
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from scipy.spatial import distance
import time
import uuid
import json
from imageio import imread
import random

from multiprocessing import Pool, Process

import pickle
import mysql.connector

import numpy as np

from FaceLoader import FaceLoader
from SocialProcessor import SocialProcessor

import vk_api

%matplotlib inline

Using TensorFlow backend.


In [3]:
conn = mysql.connector.connect(user='face', host="localhost", password="Hr3mI3PM5JlFWrub", database='global_scrape', port=8889, collation="utf8mb4_general_ci")

In [4]:
model_path = 'keras-facenet-master/model/keras/model/facenet_keras.h5'
model = load_model(model_path)

/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
def aunt_vk():
    return input(), True
def capt_vk(captcha):
    print(captcha.get_url())
    inp = input()
    captcha.try_again(inp)

In [6]:
credentials = {}
with open('credentials.json', 'r') as file:
    credentials = json.load(file)

In [7]:
vk_session = vk_api.VkApi(credentials['login'], credentials['password'], auth_handler=aunt_vk, captcha_handler=capt_vk)
vk_session.auth()

api = vk_session.get_api()

In [8]:
with open('vk_already.pcl', 'rb') as file:
    ids_already = pickle.load(file)

In [21]:
ids_scrape = set([185313244])
ids_scrape = (ids_scrape.union(set(api.friends.get(user_id=185313244)['items']))) - ids_already

In [27]:
ids_already = set()

In [24]:
processor = SocialProcessor(conn, model)

In [25]:
g_list = list(ids_scrape)
random.shuffle(g_list)
i = g_list[0]
add_set = set()
while True:
    print("To be processed:", len(g_list))
    print("Deeper level:", len(add_set))
    print(i, ':')
    ids_already.add(i)
    with open('vk_already.pcl', 'wb') as file:
        pickle.dump(ids_already, file)
    processor.processVkUser(api, i)
    with open('vk_already.pcl', 'rb') as file:
        ids_already = pickle.load(file)
    try:
        add_set = add_set.union(set(api.friends.get(user_id=i)['items']))
    except:
        pass
    ids_scrape = ids_scrape - ids_already
    if len(ids_scrape) == 0:
        ids_scrape = ids_scrape.union(add_set)
        add_set = set()
    g_list = list(ids_scrape)
    random.shuffle(g_list)
    i = g_list[0]

To be processed: 236
Deeper level: 0
169903365 :


100%|██████████| 38/38 [00:22<00:00,  1.70it/s]


Added 3 rows
To be processed: 233
Deeper level: 214
237964517 :


100%|██████████| 39/39 [00:17<00:00,  2.23it/s]


DatabaseError: 2006 (HY000): MySQL server has gone away

In [12]:
with open('vk_already.pcl', 'wb') as file:
        pickle.dump(ids_already, file)